# Customizations
* Custom Loss Functions
* Saving and loading models
* Custome Activation, initializers, regularizers and contraints

In [1]:
# Choose Python 3 runtime
#!pip install --upgrade tensorflow
import tensorflow as tf
tf.__version__

'2.0.0'

In [0]:
from keras.datasets import boston_housing
from tensorflow import keras

(X_train, y_train), (X_test, y_test) = boston_housing.load_data()

inputs = keras.Input(shape=(13,))
x = keras.layers.Dense(4, activation='relu')(inputs)
y = keras.layers.Dense(4, activation='relu')(x)
outputs = keras.layers.Dense(1, activation='sigmoid')(y)

model = keras.Model(inputs, outputs)

### Custom Loss Function
Huber loss is not defined as part of the keras API. So we can implement it ourselves. It is good for situations where we don't want to penalize large differences as much as Mean Squared Error but more than Mean Absolute Error.

In [0]:
def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss  = tf.abs(error) - 0.5
    return tf.where(is_small_error, squared_loss, linear_loss)

In [11]:
# Now you can add your huber loss function directly into you model.compile
model.compile(loss=huber_fn, optimizer="nadam")
model.fit(X_train, y_train, epochs=3)

Train on 404 samples
Epoch 1/3
404/404 [==============================] - 1s 2ms/sample - loss: 20.8972
Epoch 2/3
404/404 [==============================] - 0s 59us/sample - loss: 20.8971
Epoch 3/3
404/404 [==============================] - 0s 57us/sample - loss: 20.8970


### Saving and Loading Models That Contain Custom Components
We need to pass the custom functions as a python dictionary to be able to reload functions from saved models

In [0]:
keras.models.save_model(model, "./my_model_with_a_custom_loss.h5")

In [0]:
loaded_model = keras.models.load_model("my_model_with_a_custom_loss.h5",
                                custom_objects={"huber_fn": huber_fn})

In [20]:
loaded_model.fit(X_train, y_train, epochs=3)

Train on 404 samples
Epoch 1/3
404/404 [==============================] - 1s 1ms/sample - loss: 20.8969
Epoch 2/3
404/404 [==============================] - 0s 71us/sample - loss: 20.8968
Epoch 3/3
404/404 [==============================] - 0s 70us/sample - loss: 20.8967


### Custom Activation Functions, Initializers, Regularizers, and Constraints


In [0]:
def my_relu(z): # return value is just max of 0 and z
    return tf.maximum(0.,z)

def my_glorot_initializer(shape, dtype=tf.float32):
    stddev = tf.sqrt(2. / (shape[0] + shape[1]))
    return tf.random.normal(shape, stddev=stddev, dtype=dtype)

def my_l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01 * weights))

def my_positive_weights(weights): # return value is just tf.nn.relu(weights)
    return tf.where(weights < 0., tf.zeros_like(weights), weights)

In [0]:

from keras.datasets import boston_housing
from tensorflow import keras

(X_train, y_train), (X_test, y_test) = boston_housing.load_data()

inputs = keras.Input(shape=(13,))
x = keras.layers.Dense(4, activation=my_relu, 
                           kernel_initializer=my_glorot_initializer,
                           kernel_regularizer=my_l1_regularizer,
                           kernel_constraint=my_positive_weights)(inputs)
y = keras.layers.Dense(4, activation=my_relu,
                           kernel_initializer=my_glorot_initializer,
                           kernel_regularizer=my_l1_regularizer,
                           kernel_constraint=my_positive_weights)(x)
outputs = keras.layers.Dense(1, activation='sigmoid')(y)

model = keras.Model(inputs, outputs)

In [65]:
model.compile(loss="mean_squared_error", optimizer="nadam")
model.fit(X_train, y_train, epochs=3)

Train on 404 samples
Epoch 1/3
404/404 [==============================] - 1s 2ms/sample - loss: 544.5567
Epoch 2/3
404/404 [==============================] - 0s 63us/sample - loss: 542.4522
Epoch 3/3
404/404 [==============================] - 0s 63us/sample - loss: 542.4497
